In [84]:
import warnings 
warnings.filterwarnings("ignore")
from sklearn.linear_model import ElasticNet

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder , PolynomialFeatures 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
train = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/train.csv')
test = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/test.csv')
sample_submission = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission.csv')

In [85]:
data
data = pd.concat([train , test] , axis= 0)
test_id = test["Id"]
data.drop(["Id"] ,axis=1, inplace=True)
data["Age"] = data["YrSold"] - data["YearBuilt"]
data["Agere"] = data["YrSold"] - data["YearRemodAdd"]
data["yaxsi"] = data["1stFlrSF"]+data["2ndFlrSF"]-data["LowQualFinSF"]
data["perbas"] = ((data["TotalBsmtSF"]-data["BsmtUnfSF"])/data["BsmtUnfSF"])*100


In [104]:
str_cols = data.select_dtypes("O").columns
num_cols = data.select_dtypes("number").columns
no_cols = ["MiscFeature" , "Fence" , "GarageCond" , "GarageQual" , "GarageFinish" , "FireplaceQu" , "BsmtFinType2" , "BsmtExposure", "BsmtCond" , "BsmtQual" , "Alley"]
for col in test.columns:
    if col != "SalePrice":
        if col in no_cols:
            data[col]=data[col].fillna("no")
        elif col in str_cols:
            data[col]=data[col].fillna(data[col].mode()[0])
        elif col in num_cols:
            data[col]=data[col].fillna(data[col].median())




TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [102]:

for col in lstr_cols)-no_cols:
    if data[col].isnull().sum()/data[col].count()>0:
        print(col ,data[col].isnull().sum()/data[col].count() ) 

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [87]:
ohe  = OneHotEncoder(drop = "first")
ohe.fit(data[str_cols])
ohe_data=ohe.transform(data[str_cols]).toarray()
ohe_df = pd.DataFrame(ohe_data , columns = ohe.get_feature_names_out())

In [88]:
if list(str_cols) in list(data.columns):
    data.drop(str_cols , inplace=True ,axis = 1)
data.index=range(len(data))

ohe_df.index=range(len(ohe_df))

In [89]:
data=pd.concat([data, ohe_df], axis=1)

In [90]:
# ct = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), a.select_dtypes(include=['number']).columns)], remainder='passthrough')
# ct.fit_transform(data)


In [91]:
# cte = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), pd.Series((list(data.select_dtypes(include=['number']).columns).remove("SalePrice"))).astype("O"))], remainder='passthrough')
# cte.fit_transform(data)
# data.select_dtypes("number")

In [92]:
from sklearn.compose import ColumnTransformer

train=data[data['SalePrice'].notnull()]
test=data[data['SalePrice'].isnull()]
test.drop(['SalePrice'] , inplace=True , axis = 1)

X=train.drop('SalePrice', axis=1)
Y=train['SalePrice']

# # One-hot encode categorical vari
# # ables
ct = ColumnTransformer([('encoder', OneHotEncoder(), X.select_dtypes(include=['object']).columns)], remainder='passthrough')
X = ct.fit_transform(X)

X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size=0.3 , random_state=42)

In [93]:
def Poly_F(x , y , i , test ):
    pf = PolynomialFeatures(degree=i)
    pf.fit(x)
    x_poly = pd.DataFrame(pf.transform(x) , columns = pf.get_feature_names_out())
    pfe = PolynomialFeatures(degree=i)
    pfe.fit(test)
    test = pd.DataFrame(pfe.transform(test) , columns = pfe.get_feature_names_out())

    X_train , X_test , Y_train , Y_test = train_test_split(x_poly , y , test_size=0.3 , random_state=42)
    lr = LinearRegression()
    lr.fit(X_train , Y_train)
    # rmse_t = np.sqrt(mean_squared_error(Y_train , lr.predict(X_train)))
    # rmse= np.sqrt(mean_squared_error(Y_test , lr.predict(X_test)))
    return lr.predict(test)

In [94]:
def enet(x , y , i , j):
    enet = ElasticNet(alpha= i  , l1_ratio=j)
    X_train , X_test , Y_train , Y_test = train_test_split(x , y , test_size=0.25 , random_state=42)
    enet.fit(X_train , Y_train)
    rmse_t = np.sqrt(mean_squared_error(Y_train , enet.predict(X_train)))
    rmse= np.sqrt(mean_squared_error(Y_test , enet.predict(X_test)))
    return rmse_t , rmse , i , j

In [95]:
best_enet = {}
best_enet = dict(best_enet)
best_enety = {}
best_enety = dict(best_enety)
# for i in [1,20,80]:
#     for j in [1,20,80]:
#         a , b , c , d= enet(X ,Y ,i/100 , j/100)
#         best_enet[f" i {c} , j {d}"] = b
#         best_enety[f" i {c} , j {d}"] = a


In [96]:
# print(print(min(best_enet, key=best_enet.get)))
# print(print(min(best_enety, key=best_enety.get)))

In [97]:
#KNN - 0.39
#knn = KNeighborsClassifier()
#knn.fit(X_train , Y_train)
for col in data:
    if data[col].isnull().sum()/data[col].count()>0:
        print(col ,data[col].isnull().sum()/data[col].count() )

SalePrice 0.9993150684931507
perbas 0.027816901408450705


In [98]:
#Poly degree 2
lr = LinearRegression()
lr.fit(X , Y)


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=0.2, l1_ratio=0.7)
enet.fit(X , Y)

NameError: name 'X' is not defined

In [ ]:


filename = "/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission2.csv"
import os

test_ohe = ct.transform(test)

test_pred = enet.predict(test_ohe)

sample_submission['SalePrice'] = pd.Series(test_pred)

if os.path.exists(filename):
    os.remove(filename)
sample_submission.to_csv(filename, index=False) 
sample_submission.head()

,Id,SalePrice
0,1461,105283.087437
1,1462,145530.932080
2,1463,177076.219438
3,1464,189441.748577
4,1465,210815.814608
